# Chapter 4: Scaling with the compute layer

##### Example 1

In [ ]:
import os
from metaflow import FlowSpec, step

Why isn't the value of the `global_variable` in the `end` step equal to 42?    

**Explain**
- Because Metaflow runs each step in a separate process, and each process will have access to the global state of the Python script when the flow is initiated (`global_state=69`)
- As a result, changes to global variables in one step do not propagate to other steps, since each step has its own isolated process with its own memory space.

In [ ]:
global_value = 69

In [ ]:
class Flow(FlowSpec):
    @step
    def start(self):
        global global_value
        global_value = 42
        print("process ID is ", os.getpid())
        print("global value is ", global_value)
        self.next(self.end)

    @step
    def end(self):
        print("process ID is ", os.getpid())
        print("global value is ", global_value)

##### Example 2

In [ ]:
def train(): pass

In [ ]:
train

<function __main__.train()>

Write a flow to execute only the `train` function on AWS batch with 10GB of memory (10.000).    

In [ ]:
from metaflow import FlowSpec, step, batch

In [ ]:
class Flow(FlowSpec):
    @step
    def start(self):
        self.next(self.train)
    
    @batch(memory=10000)
    @step
    def train(self):
        train()
        self.next(self.end)
    
    @step
    def end(self): pass

In [ ]:
Flow

__main__.Flow

##### Example 3

In [3]:
class Calculator:
    def calculate(self):
        result = 1 / 0
        return result

In [2]:
from metaflow import FlowSpec, step, catch

In [5]:
class CalculatorFlow(FlowSpec):
    @step
    def start(self): self.next(self.calculate)
    
    @catch(var="division_error")
    @step
    def calculate(self):
        self.result = 1 / 0
        self.next(self.end)
    
    @step
    def end(self): pass